## De Bruijn Graph from k-mers Problem

Construct the de Bruijn graph from a collection of k-mers.

Given: A collection of k-mers Patterns.

Return: The de Bruijn graph DeBruijn(Patterns), in the form of an adjacency list.

In [14]:
from collections import defaultdict
import collections

def make_string(ls):
    ls = sorted(ls)
    string = ''
    for i in ls:
        string+= str(i) +','
    return string[:-1]

infile = open('io/rosalind_ba3e.txt','r')

nodes = defaultdict(list)

for kmer in infile:
    kmer = kmer.strip()
    a = kmer[:-1]
    b = kmer[1:]
    nodes[a].append(b)
    #nodes[b].append(a)
    
nodes = collections.OrderedDict(sorted(nodes.items()))

outfile = open("io/5_output.txt", "w")

for key in nodes:
    #print("%s -> %s" % (key, make_string(nodes[key])))
    outfile.write("%s -> %s\n" % (key, make_string(nodes[key])))

outfile.close()

## Eulerian Path Problem

Find an Eulerian path in a graph.

Given: A directed graph that contains an Eulerian path, where the graph is given in the form of an adjacency list.

Return: An Eulerian path in this graph.

In [39]:
from collections import defaultdict
import random as rd

# process input
infile = open('io/rosalind_ba3g.txt','r')
graph = defaultdict(set)

for adjlist in infile:
    items = adjlist.split('->')
    a = items[0].strip()
    b = items[1].strip().split(',')
    graph[a] = set(b)

def find_endpoint(graph):
    degrees = defaultdict(int)
    for node in graph:
        # subtract out degree
        degrees[node] -= len(graph[node])
        for edge in graph[node]:
            # add in degree
            degrees[edge] += 1
    start = []
    end = []
    for node in degrees:
        if degrees[node] < 0:
            # out > in = startnode
            start.append(node)
        elif degrees[node] > 0:
            # in > out = endnode
            end.append(node)
    if len(start) != 1 or len(end) != 1:
        raise Exception("Error :(")
    return start[0], end[0]

def find_cycle(graph, start):
    cycle = [start]
    node = start
    while True:
        edges = graph[node]
        will_visit = rd.sample(edges, 1)[0]
        if len(edges) > 1:
            graph[node] = edges - set([will_visit])
        else:
            del(graph[node])
        cycle.append(will_visit)
        node = will_visit
        if node == start:
            break
    return cycle, graph

def eulerian_path(graph):
    start, end = find_endpoint(graph)
    
    # add edge to connect end -> start (to make cyclic)
    if end in graph:
        graph[end].add(start)
    else:
        graph[end] = set([start])

    # hierholzer's algorithm
    cycle, remain = find_cycle(graph, start)
    
    while remain:
        for index, new_start in enumerate(cycle):
            if new_start in remain:
                new_cycle, remain = find_cycle(remain, new_start)
                cycle = cycle[:index] + new_cycle + cycle[index+1:]
                break
        else:
            raise Exception("Error :(")
            
    # not include the end point twice (same as start == cylic)
    return cycle[:-1]

outfile = open("io/6_output.txt", "w")
outfile.write('->'.join(eulerian_path(graph)))
outfile.close()

## String Reconstruction from Read-Pairs Problem

Reconstruct a string from its paired composition.

Given: Integers k and d followed by a collection of paired k-mers PairedReads.

Return: A string Text with (k, d)-mer composition equal to PairedReads. (If multiple answers exist, you may return any one.)

In [85]:
from collections import defaultdict

infile = open('io/rosalind_ba3j.txt','r')

vertex_name = defaultdict(str)
reverse_vertex_name = defaultdict(str)
graph = defaultdict(set)

k,d = [int(e) for e in infile.readline().split()]

now_id = 0
count_pair = 0

for pair in infile:
    kmers = pair.strip().split('|')
    edgeA = kmers[0].strip()
    edgeB = kmers[1].strip()
    vA_1 = edgeA[:-1]
    vA_2 = edgeA[1:]
    vB_1 = edgeB[:-1]
    vB_2 = edgeB[1:]
    
    vertexA = vA_1 + '|' + vB_1
    vertexB = vA_2 + '|' + vB_2
    edge = pair.strip()
    
    if vertexA not in reverse_vertex_name:
        vertex_name[str(now_id)] = vertexA
        reverse_vertex_name[vertexA] = str(now_id)
        now_id += 1
    if vertexB not in reverse_vertex_name:
        vertex_name[str(now_id)] = vertexB
        reverse_vertex_name[vertexB] = str(now_id)
        now_id += 1
    graph[reverse_vertex_name[vertexA]].add(reverse_vertex_name[vertexB])
    count_pair += 1  
    
def find_endpoint(graph):
    degrees = defaultdict(int)
    for node in graph:
        # subtract out degree
        degrees[node] -= len(graph[node])
        for edge in graph[node]:
            # add in degree
            degrees[edge] += 1
    start = []
    end = []
    for node in degrees:
        if degrees[node] < 0:
            # out > in = startnode
            start.append(node)
        elif degrees[node] > 0:
            # in > out = endnode
            end.append(node)
    if len(start) != 1 or len(end) != 1:
        raise Exception("Error :(")
    return start[0], end[0]

def find_cycle(graph, start):
    cycle = [start]
    node = start
    while True:
        edges = graph[node]
        will_visit = rd.sample(edges, 1)[0]
        if len(edges) > 1:
            graph[node] = edges - set([will_visit])
        else:
            del(graph[node])
        cycle.append(will_visit)
        node = will_visit
        if node == start:
            break
    return cycle, graph

def eulerian_path(graph):
    start, end = find_endpoint(graph)
    
    # add edge to connect end -> start (to make cyclic)
    if end in graph:
        graph[end].add(start)
    else:
        graph[end] = set([start])

    # hierholzer's algorithm
    cycle, remain = find_cycle(graph, start)
    
    while remain:
        for index, new_start in enumerate(cycle):
            if new_start in remain:
                new_cycle, remain = find_cycle(remain, new_start)
                cycle = cycle[:index] + new_cycle + cycle[index+1:]
                break
        else:
            raise Exception("Error :(")
            
    # not include the end point twice (same as start == cylic)
    return cycle[:-1]


gnome_length = count_pair + 2*k + d - 1
path = eulerian_path(graph)

gnome = vertex_name[path[0]].split('|')[0]

for p in path[1:]:
    gnome += vertex_name[p].split('|')[0][-1]
    
left_gnome = ''
j = len(path)-1

while len(gnome+left_gnome) < gnome_length:
    p = path[j]
    left_gnome = vertex_name[p].split('|')[1][-1] + left_gnome
    j -= 1
        
gnome += left_gnome

print(gnome_length)
print(len(gnome))
#print(gnome)

outfile = open("io/7_output.txt", "w")
outfile.write(gnome)
outfile.close()

1450
1450


In [83]:
a = open('io/output_7_2.txt','r')
b = open('io/7_output.txt','r')
aa = a.readline().strip()
bb = b.readline().strip()
print(len(aa),len(bb))
print(aa == bb)

6000 6000
True


# Suffix Tree Construction Problem

Construct the suffix tree of a string.

Given: A string Text.

Return: The strings labeling the edges of SuffixTree(Text). (You may return these strings in any order.)

In [3]:
input_str = open('io/input_8.txt','r').read()
suffix = list()
for i in range(len(input_str)-1,0,-1):
    suffix.append(input_str[i:])
output = open('io/output_8.txt','w')
for s in suffix:
    output.write(s+'\n')
output.close()

# Burrows-Wheeler Transform Construction Problem

Construct the Burrows-Wheeler transform of a string.

Given: A string Text.

Return: BWT(Text).

In [9]:
def bwt(string):
    matrix = [string]
    for i in range(len(string) - 1):
        matrix.append(string[i+1:] + string[:i+1])
    matrix.sort()
    return ''.join([x[-1] for x in matrix])

print('\n',bwt(input().strip()))

AAGTTATTCTCCACCTCGGTCGCAAGTTTTACTCCCAAGCTTTGACGAACTTTGCGAGAACTTACCGTGTGCGCGAGATTCACTCGAGGCGCGTACTCTTATGACCACCGAATTTTAGCTAAACGTTTGTGATAGACCTGTTAGTAAGGCCGTTTATTACGACTATCCCTGGATGTACTTGGGCCATCGGCCACGCAGAAAATGCCTACCACGAAAAAAGGAGTGAACACAGATAGACCCCCCTGGTCTTCCCATGGGCAGGAGGGACACGCGCGCGTACCATGCTGCGCTTTAGGACGCAGAGTATATGTCCGTCCTAAGAGCCAGCCAGGCCGGATGTTCTTAGCATTTGATCGCATAGTGTGTTGGTTTCGCCGGGCCAGGATAGCCGTCGTTTCCAGTTGCTAACATGCGCCGCACTGGGATCTTATATAAAATCTTTCTCGCCCGGAAGAAGAGTACAGGGGGTACCGTGGGGTATAATAGCATTGCGTTAGATGGCCGTTAGGTGGGCCGGAAAGTTGAGCTATTTTTTCTCCTCTCCGTCTTTAGCTCTTCTCGCCATTCACTATGCCTTGCAGGGTCCTATTGTTGCGGCCCACATCTCTCATACGGAGTAAATTCTAGAGTATTAACTTCCCGATCACTTACGGACACACTGTCACGGCCTTACGCTGCTTAATACACAAAGCCAGTCTAATAACACTACGAATACGTGTAGTGGGACTTAGCCTCCCGTTTGACCAGCTTCTATGACTTGTGGTCTTAACTTGGTAAAAAACGTTAGAGTTGTGGCTGTTGCGTAGCAGCCAACATTATCTGCTTATAGTTCGGACCTGCTAACTGCGTTAGATCCATCGCACGCATAAAGGGCCCCCTGGGTCCGGAGTGGATGTCACGCTACTA$

 ATTGAAAATGATCATGAATGTTCAATGTTGGTGACATA$ACTTGTAAAGCTGAGACCTCAAGTGTGCTTCGGCGTTCGCCCTTTCTAATC

In [47]:
def suffixes(string):
    for idx in range(len(string)):
        yield string[idx:]

def create_suffix_trie(text, maxdepth):
    trie = {}
    maxnode = 1

    for idx, suffix in enumerate(suffixes(text)):
        node = 1
        for char in suffix[:maxdepth] + '$':
            if node not in trie:
                trie[node] = {}
            if char == '$':
                # careful: this is not a node value, but the suffix location
                trie[node]['$'] = idx
            elif char in trie[node]:
                node = trie[node][char]
            else:
                maxnode += 1
                trie[node][char] = maxnode
                node = maxnode

    return trie

in_ = 'ATAAATG$'
#in_ = input().strip()
trie = create_suffix_trie(in_,100)
#print(output_trie(trie))

for t in trie:
    print(trie[t])
    continue
    row = ''
    for key in trie[t]:
        row += key
    print(row)

{'A': 2, 'T': 9, 'G': 23, '$': 7}
{'T': 3, 'A': 15}
{'A': 4, 'G': 21}
{'A': 5}
{'A': 6}
{'T': 7}
{'G': 8}
{'$': 0}
{'A': 10, 'G': 22}
{'A': 11}
{'A': 12}
{'T': 13}
{'G': 14}
{'$': 1}
{'A': 16, 'T': 19}
{'T': 17}
{'G': 18}
{'$': 2}
{'G': 20}
{'$': 3}
{'$': 4}
{'$': 5}
{'$': 6}


In [29]:
def get_pair(char, char_counts):
    if char in char_counts:
        new_char_count = char_counts[char] + 1
    else:
        new_char_count = 0
    return (char, new_char_count), new_char_count

def bwt_inverse(last_col):
    first_col = ''.join(sorted(last_col))
    last_char_counts, first_char_counts, next_char = {}, {}, {}

    for i in range(len(last_col)):
        last = last_col[i]
        first = first_col[i]
        
        last_pair, last_char_counts[last] = get_pair(last, last_char_counts)
        first_pair, first_char_counts[first] = get_pair(first, first_char_counts)
        next_char[last_pair] = first_pair

    result = ''
    it = next_char[('$', 0)]
    while it[0] != '$':
        result += it[0]
        it = next_char[it]

    return result + '$'

#in_ = 'TTCCTAACG$A'
#print('\n',bwt_inverse(in_).strip())
print('\n',bwt_inverse(input().strip()))

GCTAGGCGGCATGCCACACTCATCCAGTGTCGTACGGCTAACAGTACCGACCGGCGAAGTGAGGTAATCGGCATCAGTACGCCGGCCACCCGATTTTTGAGTCGGCCGCTAGTTGCACAGTCCCTAAATGATCAAAGGCCTCCTATGGCTGC$TTCTCGCACTGGATCACTCAGGCCAAGAGCGACTACACAAAGACGCTTTTTCCGTTCAGTCCCGGGAACCAGCTGGCCTTGGTTTTCGGCCTAAGTCAACTTGGGGCCCCGCTAACCTCTGTGTCTGAGCACGTGGGTTCCGTGTCATACCGAGGACTTTAGATAAAACTTCAGCATCAGCTGGATTTAAAATTAATAGTTTCGCTAGTGCCCTTTATGGCCAGACTAAAATTGGGGAATAAGGGTTCCTAACTGGCAACAGGAAACGCGCCAGCCTTTGAACCACCGCTCGCTGCGCGCCCTAACTGAGGATAGGCTCGGCTCCTTGTACCCCAGTTCATGCACCTAGTATCAACCCCGGTAATACGCCAGTAAAATTTCAATGCATGTATCACTGCATAATCCAGGGACGTCCAGCGGTAACTTCCATGGTGCCATAGCCGATATAATAAGCGTATGTTCCATAGGCCCACCCAGAAATGGAGGGAAGTGAGGTGTATTCTGTGTAGTGTTCGTGGACGAGGACTAGACAGACGAGTCTGGCCTCGCGACAAATGGAATTCTTGCAAAAATAGATTTTGTTGTGTACATAGGATTACAGATCTTCTTGCTACGTAATGTTACCTCGCCAGTAAAAGACTGCCACATCGCCTTTTTGAGTAAGCTTCTGCTGCCGTC

 AGTTTTGGGAGGTGCATCCACGCCTGGGGTGTATCCCAACTTGGAATAGATACTGTCACGTTTGGTCAGACAGGCTTATAAGGACGTACGCAGCCCCATAAGGGCGGCTCCAACGCAGCGACAGTGAAGGGATCGCCATTTAGCGTGCACAGTACCGGGTAAGCTT

In [ ]:
def get_pair(char, char_counts):
    if char in char_counts:
        new_char_count = char_counts[char] + 1
    else:
        new_char_count = 0
    return (char, new_char_count), new_char_count

def bwt_matching(last_col, pattern):
    first_col = ''.join(sorted(last_col))
    last_char_counts, first_char_counts, first_col_row, last_to_first = {}, {}, {}, {}

    for i in range(len(first_col)):
        first = first_col[i]
        first_pair, first_char_counts[first] = get_pair(first, first_char_counts)  
        first_col_row[first_pair] = i

    for i in range(len(last_col)):
        last = last_col[i]
        last_pair, last_char_counts[last] = get_pair(last, last_char_counts)
        last_to_first[i] = first_col_row[last_pair]

    top, bottom = 0, len(last_col) - 1
    while top <= bottom:
        if pattern:
            pattern, char = pattern[:-1], pattern[-1]
            top_index, bottom_index = None, -1
            found = False
            for index in range(top, bottom + 1):
                if last_col[index] == char:
                    found = True
                    if top_index == None:
                        top_index = index
                    if index > bottom_index:
                        bottom_index = index

            if found:
                top = last_to_first[top_index]
                bottom = last_to_first[bottom_index]
            else:
                return 0
        else:
            return bottom - top + 1
        
last_col = input().strip()
pattern = input().strip().split()
results = list()

for p in pattern:
    results.append(str(bwt_matching(last_col, p)))

print('\n',' '.join(results))

# Implement RandomizedMotifSearch

Given: Positive integers k and t, followed by a collection of strings Dna.

Return: A collection BestMotifs resulting from running RandomizedMotifSearch(Dna, k, t) 1000 times. Remember to use pseudocounts!

In [24]:
import random

global dna_table, dna_reverse
dna_table = {'A' : 0, 'C' : 1, 'T' : 2, 'G' : 3}
dna_reverse = { 0 : 'A', 1 : 'C', 2 : 'T', 3 : "G"}


def cal_score(motif):
    p = profile(motif)
    score = 0
    for i in range(len(p)):
        score += ( 4 + t - max(p[i]))
    return score

def new_motif(p, m):
    newMotif = list()
    for a in range(len(m)):
        max_ = 0
        j = 0
        for c in range(len(m[a])-k+1):
            i = 0
            sum_ = 1
            for b in range(c,c+len(p),1):
                sum_ *= (p[i][dna_table[m[a][b]]])
                i += 1
            if max_< sum_:
                max_ = sum_
                j = c
        newMotif.append(m[a][j:j+k])
    return newMotif

def profile(motif):
    p = list()
    for j in range(len(motif[0])):
        for i in range(len(motif)):
            if i == 0:
                p.append([1, 1, 1, 1])
                p[j][dna_table[motif[i][j]]] += 1
            else:
                p[j][dna_table[motif[i][j]]] += 1
    return p

def randomizedMotifSearch(dna, k, t):
    motif = list()
    for a in range(0,t,1):
        r = random.randrange(0, len(dna[0]) - k)
        motif.append(dna[a][r:r + k])
    bestMotif = list(motif)
    while True:
        p = profile(motif)
        motif = new_motif(p, dna)
        if cal_score(motif) < cal_score(bestMotif):
            bestMotif = list(motif)
        else:
            break
    return bestMotif


DNA = list()
DNA = open('io/input_12.txt').readlines()
DNA = [e.strip() for e in DNA]

k = int(DNA[0].split()[0])
t = int(DNA[0].split()[1])

DNA = DNA[1:]

now = list()
ans = randomizedMotifSearch(DNA,k,t)

for _ in range (0,999):
    now = randomizedMotifSearch(DNA, k, t)
    if cal_score(ans) > cal_score(now):
        ans = now
for e in ans:
    print(e)

TTGATTGTAGCACAG
TCGACACTAGGATGG
GGACTCCTAGGATGG
TCGCGTGTAGGATGG
TCGCTCCTAACTTGG
TCGCTCCTGTCATGG
TCGCTCCTAGTTGGG
TCGCTCCTAGGTACG
TCGCTGGCAGGATGG
TCGCTCCTAGGAGAA
ATGCTCCTAGGATGA
TCGCTGTAAGGATGG
TCGCTCGAGGGATGG
TCGAAGCTAGGATGG
TACTTCCTAGGATGG
TCGCTCCCTTGATGG
CCGCTCCTAGGATAT
TCCGCCCTAGGATGG
TCGCTCGCTGGATGG
TCGCATGTAGGATGG


# Implement GibbsSampler

Given: Integers k, t, and N, followed by a collection of strings Dna.

Return: The strings BestMotifs resulting from running GibbsSampler(Dna, k, t, N) with 20 random starts. Remember to use pseudocounts!

In [8]:
import random

global dna_table, dna_reverse
dna_table = {'A' : 0, 'C' : 1, 'T' : 2, 'G' : 3, 'X' : 4}
dna_reverse = { 0 : 'A', 1 : 'C', 2 : 'T', 3 : 'G', 4 : 'X'}


def cal_score(motif):
    p = profile(motif)
    score = 0
    for i in range(len(p)):
        score += ( 4 + t - max(p[i]))
    return score

def profile(motif):
    p = list()
    for j in range(len(motif[0])):
        for i in range(len(motif)):
            if i == 0:
                p.append([1, 1, 1, 1, 0])
                if dna_table[motif[i][j]] != 'X':
                    p[j][dna_table[motif[i][j]]] += 1
            elif motif[i][j] == 'X':
                continue
            else:
                p[j][dna_table[motif[i][j]]] += 1
    return p

def new_motif(p, m):
    probMot = list()
    i = 0
    for a in range(len(m)-k+1):
        c = 0
        sum_ = 1        
        for b in range(i,i+k):
            sum_*=(p[c][dna_table[m[b]]])
            c += 1
        probMot.append(sum_)
        i += 1
    return probMot

def gibbsSampler(dna, k, t,N):
    motif = []
    for a in range(0, t, 1):
        r = random.randrange(0, len(dna[0]) - k+1)
        motif.append(dna[a][r:r + k])
    print(motif)
    bestMotif = list(motif)
    for j in range(0,N):
        i = random.randrange(0,t,1)
        temp = dna[i]
        motif[i] = 'X'*k
        pf = profile(motif)
        prob = new_motif(pf,temp)
        idx = random.choices(list(range(0, len(temp)-k+1)), prob)
        motif[i] = dna[i][idx[0]:idx[0]+k]
        if cal_score(motif) < cal_score(bestMotif):
            bestMotif = list(motif)
    return bestMotif


DNA = list()
DNA = open('io/input13_t.txt').readlines()
DNA = [e.strip() for e in DNA]

DNAs = []
for dna in DNA:
    if dna[0] != '>':
        DNAs.append(dna.upper())

# k = int(DNA[0].split()[0])
# t = int(DNA[0].split()[1])
# N = int(DNA[0].split()[2])
k=8 
t=5 
N=100
# DNA = DNA[1:]

DNA = DNAs
now = list()
ANS = gibbsSampler(DNA,k,t,N)
for _ in range (0,1):
    now = gibbsSampler(DNA, k, t, N)
    if cal_score(ANS) > cal_score(now):
        ANS = now
for e in ANS:
    print(e)

['CATATATA', 'TAGTGCGG', 'TAGGCAGA', 'TAAATAGG', 'AGAAACTG']
['CACTGAAT', 'AGTGCGGG', 'AATAGATA', 'TAAACATA', 'TCAAGAAA']
CATATATA
CATATATA
CATAAATA
CATAAATA
CATAAATA


# Global Alignment with Scoring Matrix
Given: Two protein strings s
s
 and t
t
 in FASTA format (each of length at most 1000 aa).

Return: The maximum alignment score between s
s
 and t
t
.

In [25]:
input_path = 'io/input_15.txt'
BLOSUM62_path = 'data/BLOSUM62.txt'
gap = -5

with open(BLOSUM62_path, 'r') as f:
    lines = f.read().strip().split('\n')

BLOSUM62 = [lines[0].split()] + [l[1:].split() for l in lines[1:]]

def read_FASTA(path):
    read = []
    with open(path, 'r') as f:
        lines = f.read().strip().split('\n')
    now = ''
    for each in lines:
        if each[0] != '>':
            now += each
        elif len(now) > 0 and each[0] == '>':
            read.append(now)
            now = ''
    read.append(now)
    return read

def get_score(scoring_matrix, a, b):
    i = scoring_matrix[0].index(a) + 1
    j = scoring_matrix[0].index(b)
    return int(scoring_matrix[i][j])

s, t = read_FASTA(input_path) 

S = [[0 for j in range(len(t)+1)] for i in range(len(s)+1)]

for i in range(1, len(s)+1):
    S[i][0] = i * gap
for j in range(1, len(t)+1):
    S[0][j] = j * gap

for i in range(1, len(s)+1):
    for j in range(1, len(t)+1):
        S[i][j] = max([ S[i-1][j-1] + get_score(BLOSUM62, s[i-1], t[j-1]),S[i-1][j] + gap, S[i][j-1] + gap ])
        
print(S[-1][-1])

1879


# Local Alignment with Scoring Matrix
Given: Two protein strings s
s
 and t
t
 in FASTA format (each having length at most 1000 aa).

Return: A maximum alignment score along with substrings r
r
 and u
u
 of s
s
 and t
t
, respectively, which produce this maximum alignment score (multiple solutions may exist, in which case you may output any one).

In [33]:
input_path = 'io/input_16.txt'
PAM250_path = 'data/PAM250.txt'
gap = -5

with open(PAM250_path, 'r') as f:
    lines = f.read().strip().split('\n')

PAM250 = [lines[0].split()] + [l[1:].split() for l in lines[1:]]

def read_FASTA(path):
    read = []
    with open(path, 'r') as f:
        lines = f.read().strip().split('\n')
    now = ''
    for each in lines:
        if each[0] != '>':
            now += each
        elif len(now) > 0 and each[0] == '>':
            read.append(now)
            now = ''
    read.append(now)
    return read

def get_score(scoring_matrix, a, b):
    i = scoring_matrix[0].index(a) + 1
    j = scoring_matrix[0].index(b)
    return int(scoring_matrix[i][j])

s, t = read_FASTA(input_path) 

S = [[0 for j in range(len(t)+1)] for i in range(len(s)+1)]
T = [[3 for j in range(len(t)+1)] for i in range(len(s)+1)]

best = 0
best_pos = (0, 0)

for i in range(1, len(s)+1):
    for j in range(1, len(t)+1):
        cost = [ S[i-1][j-1] + get_score(PAM250, s[i-1], t[j-1]), S[i-1][j] + gap, S[i][j-1] + gap, 0 ]
        S[i][j] = max(cost)
        T[i][j] = cost.index(S[i][j])
        if S[i][j] >= best:
            best = S[i][j]
            best_pos = (i, j)

i, j = best_pos
k, l = s[:i], t[:j]

while T[i][j] != 3 and i*j != 0:
    if T[i][j] == 0:
        i -= 1
        j -= 1
    elif T[i][j] == 1:
        i -= 1
    elif T[i][j] == 2:
        j -= 1

print(best)
print(k[i:])
print(l[j:])

23
MEANLYPRTEINSTRIN
LEASANTLYEINSTEIN


# Decoding Problem

Given: A string x, followed by the alphabet Σ from which x was constructed, followed by the states States, transition matrix Transition, and emission matrix Emission of an HMM (Σ, States, Transition, Emission).

Return: A path that maximizes the (unconditional) probability Pr(x, π) over all possible paths π.

In [165]:
import numpy as np

input_path = 'io/input_17.txt'
input_file = list(open(input_path,'r'))

string = input_file[0][:-1].upper()
alphabet = dict()
alpha_read = input_file[2].split()

for i in range(len(alpha_read)):
    alphabet[alpha_read[i].upper()] = i
    
state = input_file[4].split()
tran_matrix = [[0. for _ in range(len(state))] for _ in range(len(state))]

row = 7
col = 1

for i in range(len(state)):
    for j in range(len(state)):
        tran_matrix[i][j] = float(input_file[row].split()[col])
        col += 1
    col = 1
    row += 1

emiss_matrix = [[0. for _ in range(len(alphabet))] for _ in range(len(state))]

row += 2
col = 1

for i in range(len(state)):
    for j in range(len(alphabet)):
        emiss_matrix[i][j] = float(input_file[row].split()[col])
        col += 1
    col = 1
    row += 1
    
tran_matrix = np.array(tran_matrix)
emiss_matrix = np.array(emiss_matrix)
    
# print('string:', string)
# print('alphabet:', alphabet)
# print('state:', state)
# print('tran_matrix:\n', tran_matrix)
# print('emiss_matrix:\n', emiss_matrix)

def viterbi(string, alphabet, state, tran_matrix, emiss_matrix, init_prob):
    T1 = np.zeros((len(state),len(string)))
    T2 = np.zeros(T1.shape,dtype=int)
    T1[:,0] = init_prob*emiss_matrix[:,alphabet[string[0]]]
    for i in range(1, len(string)):
        for j in range(len(state)):
            T1[j,i]= np.max(T1[:,i-1]*tran_matrix[:,j]*emiss_matrix[j,alphabet[string[i]]])
            T2[j,i]= np.argmax(T1[:,i-1]*tran_matrix[:,j]*emiss_matrix[j,alphabet[string[i]]])
    Z = np.argmax(T1, axis=0)
    result = [state[i] for i in Z]
    for i in range(T1.shape[1]-1,0,-1):
        Z[i-1] = T2[Z[i],i]
        result[i-1] = state[Z[i-1]]
    return result

init_prob = [1]*len(state)

res = viterbi(string, alphabet, state, tran_matrix, emiss_matrix, init_prob)
print(''.join(res))

BBBABBBBBBABBBBBBAABBABBABABABABBABBBBABBBBBBBBBBABBBBABBBAABABAABAABBBBBBBBABAABBABAAAABBBAABABAAAB


# Soft Decoding Problem

Given: A string x, followed by the alphabet Σ from which x was constructed, followed by the states States, transition matrix Transition, and emission matrix Emission of an HMM (Σ, States, Transition, Emission).

Return: The probability Pr(πi = k|x) that the HMM was in state k at step i (for each state k and each step i).

In [1]:
import itertools

input_path = 'io/input_18.txt'
input_file = list(open(input_path,'r'))

string = input_file[0][:-1]
alphabet = input_file[2].split()
state = input_file[4].split()
tran_matrix = [[0. for _ in range(len(state))] for _ in range(len(state))]

row = 7
col = 1

for i in range(len(state)):
    for j in range(len(state)):
        tran_matrix[i][j] = float(input_file[row].split()[col])
        col += 1
    col = 1
    row += 1

emiss_matrix = [[0. for _ in range(len(alphabet))] for _ in range(len(state))]

row += 2
col = 1

for i in range(len(state)):
    for j in range(len(alphabet)):
        emiss_matrix[i][j] = float(input_file[row].split()[col])
        col += 1
    col = 1
    row += 1
    
# print('string:', string)
# print('alphabet:', alphabet)
# print('state:', state)
# print('tran_matrix:', tran_matrix)
# print('emiss_matrix:', emiss_matrix)

def forward(string, alphabet, state , tran, emiss, pos, fixed_alpha):
    all_path = list(map(''.join, itertools.product(''.join(state), repeat=pos)))
    for i in range(len(all_path)):
        all_path[i] += fixed_alpha
    sum_score = 0.
    for path in all_path:
        score = 1.
        for i in range(len(path)):  
            score = score * emiss[state.index(path[i])][alphabet.index(string[i])]
            try:
                
                score = score * tran[state.index(path[i])][state.index(path[i+1])]
            except:
                pass
        sum_score += score
    return sum_score

def backward(string, alphabet, state , tran, emiss, pos, fixed_alpha):
    all_path = list(map(''.join, itertools.product(''.join(state), repeat=len(string)-pos-1)))
    for i in range(len(all_path)):
        all_path[i] = fixed_alpha + all_path[i]
    sum_score = 0.
    for path in all_path:
        score = 1.
        for i in range(len(path)):
            if i > 0:
                score = score * emiss[state.index(path[i])][alphabet.index(string[pos+i])]
            try:
                score = score * tran[state.index(path[i])][state.index(path[i+1])]
            except:
                pass
        sum_score += score
    return sum_score

def px(string, alphabet, state , tran, emiss):
    all_path = list(map(''.join, itertools.product(''.join(state), repeat=len(string))))
    sum_score = 0.
    for path in all_path:
        score = 1.
        for i in range(len(path)):  
            score = score * emiss[state.index(path[i])][alphabet.index(string[i])]
            try:
                score = score * tran[state.index(path[i])][state.index(path[i+1])]
            except:
                pass
        sum_score += score
    return sum_score
    

px = px(string, alphabet, state , tran_matrix, emiss_matrix)

f_str_out = ''
for s in state:
    f_str_out += s + ' '
print(f_str_out[:-1])
for i in range(len(string)):
    str_out = ''
    for s in state:
        fw = forward(string, alphabet, state , tran_matrix, emiss_matrix, i, s)
        bw = backward(string, alphabet, state , tran_matrix, emiss_matrix, i, s)
        prob = round(fw*bw/px, 4)
        str_out += str(prob) + ' '
    print(str_out[:-1])

A B
0.8068 0.1932
0.1172 0.8828
0.4957 0.5043
0.6602 0.3398
0.1698 0.8302
0.4485 0.5515
0.7216 0.2784
0.0677 0.9323
0.8641 0.1359
0.0835 0.9165


In [5]:
from collections import defaultdict
import numpy as np
import math

input_text = open('io/input_18.txt').read().split("--------\n")
text = input_text[0].strip()
all_character = input_text[1].split()
all_hidden = input_text[2].split()

print(text)
print(all_character)
print(all_hidden)


transition = np.zeros([len(all_hidden), len(all_hidden)])
for h1, i in enumerate(input_text[3].split('\n')[1:]):
    for h2, j in enumerate(i.split()[1:]):
        transition[h1][h2] = float(j)

emission = []
for i in input_text[4].split('\n')[1:]:
    dic = {}
    for c, j in zip(all_character, i.split()[1:]):
        dic[c] = float(j)
    emission.append(dic)

    
forward = np.zeros([len(text), len(all_hidden)])
for i, t in enumerate(text):
    for i1, _ in enumerate(all_hidden):
        if i > 0:
            for i2, _ in enumerate(all_hidden):
                forward[i][i1] += forward[i - 1][i2] * transition[i2][i1]
        else:
            forward[i][i1] += 1
        forward[i][i1] *= emission[i1][t]


backward = np.zeros([len(text), len(all_hidden)])
rev_text = text[::-1]
for i, t in enumerate(text[::-1]):
    for i1, _ in enumerate(all_hidden):
        if i > 0:
            for i2, _ in enumerate(all_hidden):
                backward[i][i1] += backward[i - 1][i2] * \
                    emission[i2][rev_text[i - 1]] * transition[i1][i2]
        else:
            backward[i][i1] += 1

backward = np.flip(backward, 0)
prob = forward * backward
print(" ".join(all_hidden))
for i, _ in enumerate(text):
    print(" ".join([str(round(i, 4)) for i in (prob[i] / np.sum(prob[i]))]))

zyxzyxzyzy
['x', 'y', 'z']
['A', 'B']
[[0.155 0.845]
 [0.655 0.345]]
[{'x': 0.528, 'y': 0.118, 'z': 0.354}, {'x': 0.404, 'y': 0.459, 'z': 0.137}]
A B
0.8068 0.1932
0.1172 0.8828
0.4957 0.5043
0.6602 0.3398
0.1698 0.8302
0.4485 0.5515
0.7216 0.2784
0.0677 0.9323
0.8641 0.1359
0.0835 0.9165


# Baum-Welch Learning Problem

Given: A sequence of emitted symbols x = x1 . . . xn in an alphabet A, generated by a k-state HMM with unknown transition and emission probabilities, initial Transition and Emission matrices and a number of iterations I.

Return: A matrix of transition probabilities Transition and a matrix of emission probabilities Emission that maximizes Pr(x,π) over all possible transition and emission matrices and over all hidden paths π.

In [26]:
from collections import defaultdict
import numpy as np
import math

# Read input
input_path = 'io/input_19.txt'
input_file = list(open(input_path,'r'))

epoch = int(input_file[0])
string = input_file[2][:-1]
alphabet = input_file[4].split()
state = input_file[6].split()
tran_matrix = [[0. for _ in range(len(state))] for _ in range(len(state))]

row = 9
col = 1

for i in range(len(state)):
    for j in range(len(state)):
        tran_matrix[i][j] = float(input_file[row].split()[col])
        col += 1
    col = 1
    row += 1

emiss_matrix = [[0. for _ in range(len(alphabet))] for _ in range(len(state))]

row += 2
col = 1

for i in range(len(state)):
    for j in range(len(alphabet)):
        emiss_matrix[i][j] = float(input_file[row].split()[col])
        col += 1
    col = 1
    row += 1
    
# baum_welch
for step in range(epoch):
    # E-step
    emiss_matrix_d = []
    for e_row in emiss_matrix:
        d = dict()
        for i in range(len(alphabet)):
            d[alphabet[i]] = e_row[i]
        emiss_matrix_d.append(d)
    
    text = string
    all_hidden = state
    transition = tran_matrix
    emission = emiss_matrix_d
    
    forward = np.zeros([len(text), len(all_hidden)])
    for i, t in enumerate(text):
        for i1, _ in enumerate(all_hidden):
            if i > 0:
                for i2, _ in enumerate(all_hidden):
                    forward[i][i1] += forward[i - 1][i2] * transition[i2][i1]
            else:
                forward[i][i1] += 1
            forward[i][i1] *= emission[i1][t]


    backward = np.zeros([len(text), len(all_hidden)])
    rev_text = text[::-1]
    for i, t in enumerate(text[::-1]):
        for i1, _ in enumerate(all_hidden):
            if i > 0:
                for i2, _ in enumerate(all_hidden):
                    backward[i][i1] += backward[i - 1][i2] * \
                        emission[i2][rev_text[i - 1]] * transition[i1][i2]
            else:
                backward[i][i1] += 1
 
    pi_star = []
    pi_star_star = []
    
    backward = np.flip(backward, 0)

    prob = forward * backward
    for i, _ in enumerate(text):
        prx = np.sum(prob[i])
        pi_star.append(prob[i]/prx)
   
    for i in range(len(string)-1):
        pi_star_star_mini = []
        for j in range(len(state)):
            for k in range(len(state)):
                edge_prob = forward[i][j] * tran_matrix[j][k] * emiss_matrix[k][alphabet.index(string[i+1])] * backward[i+1][k]/np.sum(prob[i])
                pi_star_star_mini.append(edge_prob)
        pi_star_star.append(pi_star_star_mini)
    

    # M-step
    T = [0.]*(len(state)**2)
    new_tran = []

    for i in range(len(pi_star_star)):
        for j in range(len(state)**2):
            T[j] += pi_star_star[i][j]

    for i in range(0,len(T),len(state)):
        row = T[i:i+len(state)]
        k = sum(row)
        mini = []
        for r in row:
            mini.append(r/k)
        new_tran.append(mini)

    tran_matrix = new_tran

    E = [0.]*(len(state)*len(alphabet))
    new_emiss = []

    for i in range(len(pi_star)):
        for j in range(len(alphabet)):
            if string[i] == alphabet[j]:
                for k in range(len(state)):
                    E[(k*len(alphabet))+j] += pi_star[i][k]
                break

    for i in range(0,len(E),len(alphabet)):
        row = E[i:i+len(alphabet)]
        k = sum(row)
        mini = []
        for r in row:
            mini.append(r/k)
        new_emiss.append(mini)

    emiss_matrix = new_emiss

# Print output

first_out = ''
for s in state:
    first_out += s + ' '
print(first_out[:-1])

for i in range(len(state)):
    tran_row = tran_matrix[i]
    out_str_row = state[i] + ' '
    for each in tran_row:
        no_pad = str(round(each,3))
        while len(no_pad) < 5:
            no_pad += '0'
        out_str_row += no_pad +' '
    print(out_str_row[:-1])

print('--------')

second_out = ''
for a in alphabet:
    second_out += a + ' '
print(second_out[:-1])

for i in range(len(state)):
    emiss_row = emiss_matrix[i]
    out_str_row = state[i] + ' '
    for each in emiss_row:
        no_pad = str(round(each,3))
        while len(no_pad) < 5:
            no_pad += '0'
        out_str_row += no_pad +' '
    print(out_str_row[:-1])

A B C D
A 0.355 0.644 0.001 0.000
B 0.000 0.565 0.435 0.000
C 0.053 0.032 0.432 0.483
D 0.838 0.015 0.073 0.074
--------
x y z
A 0.924 0.000 0.076
B 0.332 0.529 0.139
C 0.120 0.089 0.791
D 0.649 0.268 0.083


In [14]:
import itertools

input_path = 'io/input_19.txt'
input_file = list(open(input_path,'r'))

epoch = int(input_file[0])
string = input_file[2][:-1]
alphabet = input_file[4].split()
state = input_file[6].split()
tran_matrix = [[0. for _ in range(len(state))] for _ in range(len(state))]

row = 9
col = 1

for i in range(len(state)):
    for j in range(len(state)):
        tran_matrix[i][j] = float(input_file[row].split()[col])
        col += 1
    col = 1
    row += 1

emiss_matrix = [[0. for _ in range(len(alphabet))] for _ in range(len(state))]

row += 2
col = 1

for i in range(len(state)):
    for j in range(len(alphabet)):
        emiss_matrix[i][j] = float(input_file[row].split()[col])
        col += 1
    col = 1
    row += 1
    
    
emiss_matrix_d = []

for e_row in emiss_matrix:
    d = dict()
    for i in range(len(alphabet)):
        d[alphabet[i]] = e_row[i]
    emiss_matrix_d.append(d)
    
def forward(string, alphabet, state , tran, emiss, pos, fixed_alpha):
    all_path = list(map(''.join, itertools.product(''.join(state), repeat=pos)))
    for i in range(len(all_path)):
        all_path[i] += fixed_alpha
    sum_score = 0.
    for path in all_path:
        score = 1.
        for i in range(len(path)):  
            score = score * emiss[state.index(path[i])][alphabet.index(string[i])]
            try:
                
                score = score * tran[state.index(path[i])][state.index(path[i+1])]
            except:
                pass
        sum_score += score
    return sum_score

def backward(string, alphabet, state , tran, emiss, pos, fixed_alpha):
    all_path = list(map(''.join, itertools.product(''.join(state), repeat=len(string)-pos-1)))
    for i in range(len(all_path)):
        all_path[i] = fixed_alpha + all_path[i]
    sum_score = 0.
    for path in all_path:
        score = 1.
        for i in range(len(path)):
            if i > 0:
                score = score * emiss[state.index(path[i])][alphabet.index(string[pos+i])]
            try:
                score = score * tran[state.index(path[i])][state.index(path[i+1])]
            except:
                pass
        sum_score += score
    return sum_score

def px_cal(string, alphabet, state , tran, emiss):
    all_path = list(map(''.join, itertools.product(''.join(state), repeat=len(string))))
    sum_score = 0.
    for path in all_path:
        score = 1.
        for i in range(len(path)):  
            score = score * emiss[state.index(path[i])][alphabet.index(string[i])]
            try:
                score = score * tran[state.index(path[i])][state.index(path[i+1])]
            except:
                pass
        sum_score += score
    return sum_score
    

print(string, alphabet, state , tran_matrix, emiss_matrix_d)
    
# baum_welch
for step in range(1):
    px = px_cal(string, alphabet, state , tran_matrix, emiss_matrix)
    pi_star = []
    pi_star_star = []
    fw_stat = []
    bw_stat = []
    for i in range(len(string)):
        str_out = ''
        fw_stat_mini = []
        bw_stat_mini = []
        for s in state:
            fw = forward(string, alphabet, state , tran_matrix, emiss_matrix, i, s)
            bw = backward(string, alphabet, state , tran_matrix, emiss_matrix, i, s)
            fw_stat_mini.append(fw)
            bw_stat_mini.append(bw)
            prob = fw*bw/px
            str_out += str(prob) + ' '
        pi_star.append(list(float(e) for e in str_out.split()))
        fw_stat.append(fw_stat_mini)
        bw_stat.append(bw_stat_mini)
        
    print(fw_stat)
    print(bw_stat)
    break

    for i in range(len(string)-1):
        pi_star_star_mini = []
        for j in range(len(state)):
            for k in range(len(state)):
                edge_prob = fw_stat[i][j] * tran_matrix[j][k] * emiss_matrix[k][alphabet.index(string[i+1])] * bw_stat[i+1][k]/px
                pi_star_star_mini.append(edge_prob)
        pi_star_star.append(pi_star_star_mini)
    
    
    T = [0.]*(len(state)**2)
    new_tran = []

    for i in range(len(pi_star_star)):
        for j in range(len(state)**2):
            T[j] += pi_star_star[i][j]

    for i in range(0,len(T),len(state)):
        row = T[i:i+len(state)]
        k = sum(row)
        mini = []
        for r in row:
            mini.append(r/k)
        new_tran.append(mini)

    tran_matrix = new_tran

    E = [0.]*(len(state)*len(alphabet))
    new_emiss = []

    for i in range(len(pi_star)):
        for j in range(len(alphabet)):
            if string[i] == alphabet[j]:
                for k in range(len(state)):
                    E[(k*len(alphabet))+j] += pi_star[i][k]
                break

    for i in range(0,len(E),len(alphabet)):
        row = E[i:i+len(alphabet)]
        k = sum(row)
        mini = []
        for r in row:
            mini.append(r/k)
        new_emiss.append(mini)

    emiss_matrix = new_emiss

first_out = ''
for s in state:
    first_out += s + ' '
print(first_out[:-1])

for i in range(len(state)):
    tran_row = tran_matrix[i]
    out_str_row = state[i] + ' '
    for each in tran_row:
        no_pad = str(round(each,3))
        while len(no_pad) < 5:
            no_pad += '0'
        out_str_row += no_pad +' '
    print(out_str_row[:-1])

print('--------')

second_out = ''
for a in alphabet:
    second_out += a + ' '
print(second_out[:-1])

for i in range(len(state)):
    emiss_row = emiss_matrix[i]
    out_str_row = state[i] + ' '
    for each in emiss_row:
        no_pad = str(round(each,3))
        while len(no_pad) < 5:
            no_pad += '0'
        out_str_row += no_pad +' '
    print(out_str_row[:-1])

xzyyzyzyxy ['x', 'y', 'z'] ['A', 'B'] [[0.019, 0.981], [0.668, 0.332]] [{'x': 0.175, 'y': 0.003, 'z': 0.821}, {'x': 0.196, 'y': 0.512, 'z': 0.293}]
[[0.175, 0.196], [0.11022171300000001, 0.069366871], [0.00014529384712499998, 0.067152538432], [0.00013458196876701416, 0.011487834124008257], [0.006302349237092397, 0.0011561738512756365], [2.676206304470642e-06, 0.00336202461194811], [0.0018438701800236215, 0.0003278135361585773], [7.62038926723135e-07, 0.000981847419191204], [0.00011478049708288312, 6.403729705828462e-05], [1.3487323163852688e-07, 6.85363457339691e-05]]
[[5.933686789526865e-05, 0.00029738401573436643], [0.0005105463090300721, 0.00017872984667977772], [0.0029956882356845928, 0.001016133797624869], [0.00116012389456181, 0.005964143150768117], [0.01025781873341917, 0.0034794619272637607], [0.003983734540619805, 0.020422384008167586], [0.035117756061563137, 0.01195385590316868], [0.034739408613000004, 0.069913863236], [0.502329, 0.17198800000000003], [1.0, 1.0]]
A B
A 0.019 

In [166]:
# 0.1422 0.8578
# 0.8178 0.1822
# 0.0063 0.9937
# 0.0021 0.9979
# 0.942 0.058
# 0.0001 0.9999
# 0.9435 0.0565
# 0.0003 0.9997
# 0.841 0.159
# 0.0019 0.9981

#xzyyzyzyxy xzyyzyzyxy
# BABBABABAB ABAABAAAAA
# AA = 0 5
# AB = 4 2
# BA = 4 2
# BB = 1 0
# A->x = 1 2
# A->y = 0 5
# A->z = 3 1
# B->x = 1 0
# B->y = 5 0
# B->z = 0 2
